 Testing output from an api run


In [2]:
from data_models2 import Entity, Labels
import rich
from openai import AzureOpenAI
import azure.keyvault.secrets as azk
from azure.identity import DefaultAzureCredential
import os, asyncio

In [4]:
def get_key(keyname):
    vault_uri = 'https://keyvault-labeling.vault.azure.net/'
    client = azk.SecretClient(vault_uri, DefaultAzureCredential())
    secret = client.get_secret(keyname)
    key = secret.value
    return key

async def send_request(filepath:str):
    with open(filepath,'r') as f:
        contents = f.read().replace("\n", " ").replace("\t", " ").replace("\r", " ")

    key = get_key('openai-key')
    endpoint = 'https://labeling-llm-0.openai.azure.com/openai/deployments/gpt-4.1-nano/chat/completions?api-version=2025-01-01-preview'
    client = AzureOpenAI(
    azure_endpoint = endpoint,
    api_key=key,
    api_version="2024-12-01-preview"
    )
    completion = client.beta.chat.completions.parse(
        model="gpt-4.1-nano",
        messages=[
            {
                "role": "system",
                "content": """You are a Named Entity Recognition (NER) model used as an advanced ATS scanner.
                Your job is to extract words matching the specified entity types; soft skill, Capability, Personality trait, Job title. 
                Skip words not matching any of these types with well enough accuracy that a human would categorise them the same.
                Only assign labels if youre certain that they are relevant in the job market and fall under one of specified categories with well enough accuracy.
                Do not reformat or translate the text you find in the original document in any way.
                """,
            },
            {
                "role": "user",
                "content": contents,
            }
        ],
        response_format=Labels,
        n=1,

    )
    
    message = completion.choices[0].message
    if (message.refusal):
        print(f'error in request at file {filepath}:\n{message.refusal}\n\n{message}')
        return 0
    else:
        labels = Labels.model_validate(message.parsed).model_dump()['Entities']
        return (contents, labels)


obj = await send_request("/home/ronji/repos/ner-api/data_cs/17.txt")
contents = obj[0].replace("\n", " ").replace("\t", " ").replace("\r", " ") # type: ignore
labels = obj[1] # type: ignore
rich.print(obj)


(
    '         Shrnutí CHEF Vysoce organizovaný a efektivní v rychle se měnícím multitaskingovém prostředí; Dokáže 
efektivně stanovit priority a dosáhnout cílů s kreativitou, nadšením a humorem.\xa0Zkušený supervizor a školitel; 
Dokáže jasně vysvětlit informace, rozpoznat dokonalost jednotlivců a inspirovat členy týmu, aby dosáhli svého 
potenciálu a zároveň pracovali na společných cílech.      Zkušenosti 07/2009 až 07/2014 Chef Název firmy - Město, 
Stát Potraviny připravované důsledně a v souladu s recepty, porcování, vaření a plýtvání      pokyny pro kontrolu. 
Vedené směny při osobní přípravě jídel a vyřizování požadavků na základě požadavků      specifikace.         
06/2014 až 08/2014 Dělník Název firmy - Město, Stát Naložený a vyložený stavební materiál použitý pro stavbu.  
Ručně přenášené střešní krytiny po žebřících.  Důsledně přebíral další odpovědnosti a pracoval prodloužené hodiny, 
aby splnil projekt      termíny.         06/2015 až 08/2016 Laborer & Crew Manager Název společnosti - Město, Stát 
Zajišťoval plánování pro člena posádky a časy příjezdu pro klienty Udržoval bezpečné pracovní prostředí pro členy 
posádky Vedl posádky až čtyř lidí a spolupracoval na posádkách až osmi Vzdělání a školení Středoškolský diplom: 
Všeobecné vzdělání New London High School - Město, Státní IT - Management University of Wisconsin - Stout - Město, 
Stát, USA Navštěvoval jsem UW - Stout dva roky, než jsem byl finančně neschopný pokračovat ve školní docházce. I 
když jsem většinou navštěvoval kurzy všeobecného vzdělávání, měl jsem i několik kurzů informatiky a managementu.   
IT - Specialista na počítačovou podporu Fox Valley Technical College - Město , stát , USA V současné době 
navštěvuji Fox Valley Tech a mým cílem je začít dosáhnout přidruženého titulu. Až do této chvíle jsem zde byl 
zapsán na celý rok.        Dovednosti Týmový hráč, Fyzicky aktivní, společenský, Ochota učit se, Projevuje 
iniciativu       ',
    [
        {'text': 'Soft skills', 'type': 'SOFT_SKILL'},
        {'text': 'organizovaný', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'efektivní', 'type': 'CAPABILITY'},
        {'text': 'rychle se měnícím multitaskingovém prostředí', 'type': 'CAPABILITY'},
        {'text': 'stanovit priority', 'type': 'CAPABILITY'},
        {'text': 'kreativitou', 'type': 'SOFT_SKILL'},
        {'text': 'nadšením', 'type': 'SOFT_SKILL'},
        {'text': 'humorem', 'type': 'SOFT_SKILL'},
        {'text': 'zkušený', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'supervizor', 'type': 'JOB_TITLE,EDUCATION,HOBBY,INSTITUTION'},
        {'text': 'školitel', 'type': 'JOB_TITLE,EDUCATION,HOBBY,INSTITUTION'},
        {'text': 'jasně vysvětlit', 'type': 'CAPABILITY'},
        {'text': 'inspirovat', 'type': 'CAPABILITY'},
        {'text': 'potenciálu', 'type': 'CAPABILITY'},
        {'text': 'práce na společných cílech', 'type': 'CAPABILITY'},
        {'text': 'důslednost', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'kontrolu', 'type': 'CAPABILITY'},
        {'text': 'řízení směn', 'type': 'CAPABILITY'},
        {'text': 'příprava jídel', 'type': 'CAPABILITY'},
        {'text': 'vyřizování požadavků', 'type': 'CAPABILITY'},
        {'text': 'specifikace', 'type': 'CAPABILITY'},
        {'text': 'plánování pro člena posádky', 'type': 'CAPABILITY'},
        {'text': 'doprava a práce s klienty', 'type': 'CAPABILITY'},
        {'text': 'ukládání a přenášení stavebního materiálu', 'type': 'CAPABILITY'},
        {'text': 'přesnost', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'vedení posádky', 'type': 'CAPABILITY'},
        {'text': 'bezpečnost práce', 'type': 'CAPABILITY'},
        {'text': 'týmový hráč', 'type': 'SOFT_SKILL'},
        {'text': 'fyzicky aktivní', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'společenský', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'ochota učit se', 'type': 'SOFT_SKILL'},
        {'text': 'iniciativa', 'type': 'SOFT_SKILL'}
    ]
)

In [1]:
obj = (
    '''         SALES TEAM       Executive Summary     Results-focused management professional offering 25 years of 
progressive leadership experience. Transforms high-potential staff into outstanding leaders who demonstrates the 
creativity and savvy that is critical to financial and operational success.       Core Qualifications          
Operations management  Staff development  Inventory control  Compensation/benefits administration  Policy/program 
development  Staff training  Supervision and training  Sound judgment  Computer-savvy  Calm under pressure  Complex
problem solving      Microsoft applications  Forecasting  Excellent writing skills  Billing  Attention to detail  
Multi-state payroll  Exceptionally organized  Record-keeping  I-9 documentation  Accounts payable            
Professional Experience      Sales Team     Nov 2015   to   Feb 2016      Company Name   -   City  ,   State     
Delivered unparalleled customer services Greeted and assisted customers; responded to customer inquiries and 
complaints in a professional and timely manner.  Communicated product knowledge to the customer and recommend 
merchandise selections.  Trained in visual merchandising and organization.  Monitored sales activities and 
productivity Ensured register transaction where completed accurately and in accordance with policy.  Excellent 
communication, verbal, and written skills.  Strong organizational skills and ability to mulit-task in a fast paced 
environment.         Sales Team     Nov 2015   to   Jan 2016      Company Name   -   City  ,   State     Provided 
outstanding customer services Maintained displays and store appearance Ensured customer satisfaction at every state
of sale.  Opened and Closed store in accordance with Company policies.  Recommended new products and upcoming 
events.  Extensive sales and customer service experience.  Broad expertise in advertisement and promotion.         
Manager     Mar 2009   to   Jan 2015      Company Name   -   City  ,   State     Management Lead and oversaw the 
implementation of long and short term plans in accordance with strategic plans Ensured expenditures were within the
authorized annual budget Implemented operational improvements to enhance quality, production times and reduce costs
Decreased production labor hours after implementing a time study of the production line resulting in a cost savings
of approximately $20,000 per unit Conducted industry standard study on the cost of services calls and inspection 
charges resulting in tripled income of services department revenue Implemented an inventory software program to 
operate with the accounting program to streamline communication with purchasing, inventory, engineering and 
production departments Experience with Lean Six Sigma Experience supervising large number of employees Advanced 
experience using Microsoft Office suite and PowerPoint.  Project Management  Communicated effectively with 
shareholders, employees, Government authorities and stakeholders Managed client expectations by ensuring the 
highest quality of service Developed, implemented and ensured compliance of safety procedures to meet government 
guidelines Monitored employee productivity, performance and optimized employee procedures to reduce costs Monitored
and maintained operational reports to detect production issues.  Set up and management of over 60 vendor 
relationships Responsible for delivering projects against agreed scope, budget, schedule and customer expectations 
Developed production tracking and quality control systems while analyzing production Implemented new product lines 
through research and development to generate new revenue streams Collaborated with Mine Safety and Health 
Administration (MSHA), a division of the United States Department of Labor, to ensure the safety and survivability 
of miners in the event of an incident while underground Received all four of MSHA\'s approvals in Product, 
Breathable Air, Harmful Gas Removal, Gas Monitoring and Structural categories General Management Recruited, trained
and supervised human resource director, safety manager, engineers, controllers and production manager Created 
marketing materials, presentation and demonstration products for sales meetings, trade shows and consumer education
Managed all day to day facility operations Responsible for payroll oversight and related approvals.         Manager
Feb 2004   to   Jan 2009      Company Name   -   City  ,   State     Developed and operated four assisted living 
locations Obtained and maintained annual state certifications and supervised all security transportation and 
monitoring needs Managed day to day facility operations and admissions and coordinated daily services including 
nursing, dining, housekeeping, activities and maintenance functions Coordinated new resident move-ins and 
orientation and supported the admission process through tours and follow up calls to inquiries Project Management 
Monitored budget compliance and financial outcomes including labor and operational expenses and accounts receivable
resolution Responsible for payroll oversight and related approvals Oversaw accuracy and thoroughness of all 
activity documentation within the assisted living facility Kept current on all changes in the industry, 
particularly regulatory changes as they relate to Assisted Living and Dementia Care General Management Supervised 
the assisted living staff including caregivers, medication technicians, licensed nurses and activities staff 
Responsible for recruiting and onboarding of staff and all human resources duties for employees Planned, directed 
and facilitated marketing events for all facilities Conducted outside sales calls to generate inquiries 
ACHIEVEMENTS Featured in Space Safety Magazine Integrated lifesaving technologies originally developed to purify 
air for astronauts into a line of Mine Shield products Integral part of Mine Shields getting recognized as "New 
Business of the Year" Mine Shield tested for and received certification in China under my direction.         
Education      High School Diploma     1980     Nelson County High School   -   City  ,   State            
Interests    CrossFit, white water rafting, waterboarding and reading        Additional Information      PERSONAL 
INTERESTS   CrossFit, white water rafting, water boarding and reading        Skills     accounting, accounts 
receivable, streamline, Assisted Living, budget, Excellent communication, client, customer satisfaction, customer 
services, customer service experience, direction, documentation, fast, financial, General Management, Government, 
human resource, human resources, inspection, inventory, director, marketing, marketing materials, meetings, 
Microsoft Office suite, PowerPoint, nursing, Strong organizational skills, outside sales, payroll, policies, 
production manager, Project Management, promotion, purchasing, quality, quality control, recruiting, research, 
Safety, sales, Six Sigma, strategic plans, supervising, trade shows, transportation, visual merchandising, written 
skills    ''',
    [
        {'text': 'Results-focused', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'management', 'type': 'CAPABILITY'},
        {'text': 'transforms high-potential staff into outstanding leaders', 'type': 'CAPABILITY'},
        {'text': 'creativity', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'savvy', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'financial', 'type': 'CAPABILITY'},
        {'text': 'operational', 'type': 'CAPABILITY'},
        {'text': 'Staff development', 'type': 'CAPABILITY'},
        {'text': 'Inventory control', 'type': 'CAPABILITY'},
        {'text': 'Compensation/benefits administration', 'type': 'CAPABILITY'},
        {'text': 'Policy/program development', 'type': 'CAPABILITY'},
        {'text': 'Staff training', 'type': 'CAPABILITY'},
        {'text': 'Supervision', 'type': 'CAPABILITY'},
        {'text': 'Sound judgment', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'Computer-savvy', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'Calm under pressure', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'Complex problem solving', 'type': 'CAPABILITY'},
        {'text': 'Microsoft applications', 'type': 'CAPABILITY'},
        {'text': 'Forecasting', 'type': 'CAPABILITY'},
        {'text': 'billing', 'type': 'CAPABILITY'},
        {'text': 'Attention to detail', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'Multi-state payroll', 'type': 'CAPABILITY'},
        {'text': 'Exceptionally organized', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'Record-keeping', 'type': 'CAPABILITY'},
        {'text': 'I-9 documentation', 'type': 'CAPABILITY'},
        {'text': 'Accounts payable', 'type': 'CAPABILITY'},
        {'text': 'Customer services', 'type': 'SOFT_SKILL'},
        {'text': 'verbal, and written skills', 'type': 'SOFT_SKILL'},
        {'text': 'organizational skills', 'type': 'SOFT_SKILL'},
        {'text': 'multitask', 'type': 'SOFT_SKILL'},
        {'text': 'sales', 'type': 'CAPABILITY'},
        {'text': 'customer service', 'type': 'SOFT_SKILL'},
        {'text': 'brand', 'type': 'CAPABILITY'},
        {'text': 'Management', 'type': 'JOB_TITLE'},
        {'text': 'Lead', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'oversaw', 'type': 'CAPABILITY'},
        {'text': 'implemented', 'type': 'CAPABILITY'},
        {'text': 'cost savings', 'type': 'CAPABILITY'},
        {'text': 'Conducted', 'type': 'CAPABILITY'},
        {'text': 'study', 'type': 'CAPABILITY'},
        {'text': 'on the cost of services calls and inspection charges', 'type': 'CAPABILITY'},
        {'text': 'revenue', 'type': 'CAPABILITY'},
        {'text': 'Developed', 'type': 'CAPABILITY'},
        {'text': 'inventory software program', 'type': 'CAPABILITY'},
        {'text': 'streamline', 'type': 'CAPABILITY'},
        {'text': 'experience', 'type': 'CAPABILITY'},
        {'text': 'supervising', 'type': 'CAPABILITY'},
        {'text': 'advanced experience', 'type': 'CAPABILITY'},
        {'text': 'Project Management', 'type': 'CAPABILITY'},
        {'text': 'Communicated', 'type': 'CAPABILITY'},
        {'text': 'managed', 'type': 'CAPABILITY'},
        {'text': 'safety procedures', 'type': 'CAPABILITY'},
        {'text': 'Monitored', 'type': 'CAPABILITY'},
        {'text': 'Operational reports', 'type': 'CAPABILITY'},
        {'text': 'mission', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'responsibility', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'Delivery', 'type': 'CAPABILITY'},
        {'text': 'tracking', 'type': 'CAPABILITY'},
        {'text': 'collaborated', 'type': 'CAPABILITY'},
        {'text': 'safety and survivability', 'type': 'CAPABILITY'},
        {'text': 'Received', 'type': 'CAPABILITY'},
        {'text': 'creation and supervision', 'type': 'CAPABILITY'},
        {'text': 'recruiting', 'type': 'CAPABILITY'},
        {'text': 'training', 'type': 'CAPABILITY'},
        {'text': 'Supervised', 'type': 'CAPABILITY'},
        {'text': 'recruitment', 'type': 'CAPABILITY'},
        {'text': 'onboarding', 'type': 'CAPABILITY'},
        {'text': 'Marketing', 'type': 'JOB_TITLE'},
        {'text': 'Managed', 'type': 'CAPABILITY'},
        {'text': 'Operations management', 'type': 'CAPABILITY'},
        {'text': 'Customer satisfaction', 'type': 'SOFT_SKILL'},
        {'text': 'dedication', 'type': 'PERSONALITY_TRAIT'}
    ]
)

contents = obj[0].replace("\n", " ").replace("\t", " ").replace("\r", " ") # type: ignore
labels = obj[1] # type: ignore

In [2]:
from label_models import Label, Document

In [ ]:
def format_entities(doc_contents:str,labels_list:list,doc_id:int=0):
    doc = doc_contents.replace("\n", " ").replace("\t", " ").replace("\r", " ")
    entities = labels_list
    labeled_ents = []
    for entity in entities:
        content = entity['text']
        label = entity['type']
        if content not in doc_contents:
            continue
        else:
            start = doc_contents.find(content)
            end = start + len(content)
            labeled_ent = Label(start,end,label,content).to_dict()
            labeled_ents.append(labeled_ent)
    labeled_doc = Document(doc_id,doc_contents, labeled_ents).to_dict()
    return labeled_doc

test_doc = format_entities(contents,labels)
rich.print(test_doc)